In [1]:
!python -c "import torch; print(torch.__version__)"

1.12.1+cu113


In [2]:
!python -c "import torch; print(torch.version.cuda)"


11.3


In [3]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
     |████████████████████████████████| 7.9 MB 30.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.12.0+cu113.html
     |████████████████████████████████| 3.5 MB 32.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 467 kB 36.6 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.1.0.post1-py3-none-any.whl size=689859 sha256=b3a1ab97776e70ff30a70285dc87ab4ff7352044a1b65adf3e141628628ef7d7
  Stored in directory: /root/.cache/pip/wheels/d1/cb/43/f7f2e472de4d7cff31bceddadc36d634e1e545fbc17961c282
Successfully built torch-geometric


In [4]:
from torch_geometric.nn import GCN
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
import torch

In [5]:
dataset = TUDataset(root='tmp/ENZYMES', name='ENZYMES')

print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Extracting tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...



Dataset: ENZYMES(600):
Number of graphs: 600
Number of features: 3
Number of classes: 6

Data(edge_index=[2, 168], x=[37, 3], y=[1])
Number of nodes: 37
Number of edges: 168
Average node degree: 4.54
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Done!


In [6]:
dataset = dataset.shuffle()

train_dataset = dataset[:400]
test_dataset = dataset[:200]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

for step, data in enumerate(train_loader):
  print(f"Step {step +1}:")
  print('===========')
  print(f'Number of graphs in current batch: {data.num_graphs}')
  print(data)
  print()

Step 1:
Number of graphs in current batch: 64
DataBatch(edge_index=[2, 8378], x=[2147, 3], y=[64], batch=[2147], ptr=[65])

Step 2:
Number of graphs in current batch: 64
DataBatch(edge_index=[2, 7870], x=[2046, 3], y=[64], batch=[2046], ptr=[65])

Step 3:
Number of graphs in current batch: 64
DataBatch(edge_index=[2, 8474], x=[2154, 3], y=[64], batch=[2154], ptr=[65])

Step 4:
Number of graphs in current batch: 64
DataBatch(edge_index=[2, 8034], x=[2188, 3], y=[64], batch=[2188], ptr=[65])

Step 5:
Number of graphs in current batch: 64
DataBatch(edge_index=[2, 8262], x=[2186, 3], y=[64], batch=[2186], ptr=[65])

Step 6:
Number of graphs in current batch: 64
DataBatch(edge_index=[2, 6832], x=[1772, 3], y=[64], batch=[1772], ptr=[65])

Step 7:
Number of graphs in current batch: 16
DataBatch(edge_index=[2, 1882], x=[489, 3], y=[16], batch=[489], ptr=[17])



some other attempts with gym

In [7]:
!git clone https://github.com/pyg-team/pytorch_geometric.git
!cd pytorch_geometric/graphgym

Cloning into 'pytorch_geometric'...
remote: Enumerating objects: 40982, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 40982 (delta 16), reused 14 (delta 8), pack-reused 40942
Receiving objects: 100% (40982/40982), 8.49 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (30888/30888), done.


In [8]:
%ls

pytorch_geometric/  sample_data/  tmp/


In [9]:
%%shell 

. pytorch_geometric/graphgym/run_batch.sh

python3: can't open file 'configs_gen.py': [Errno 2] No such file or directory
bash: parallel.sh: No such file or directory
bash: parallel.sh: No such file or directory
bash: parallel.sh: No such file or directory
python3: can't open file 'agg_batch.py': [Errno 2] No such file or directory


CalledProcessError: ignored

attempt to manually load gcn and train it. it failed

In [10]:
model = GCN(in_channels=-1, hidden_channels=64, num_layers=6)

print(model)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.
    


def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

GCN(-1, 64, num_layers=6)


ValueError: ignored